## Load/import packages

In [ ]:
import json
import scipy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import kerastuner as kt

from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.utils import class_weight

from os import listdir
from os.path import join, splitext, normpath

# Import packages to run custom FW-RNN cell
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras import activations, initializers
from tensorflow.python.keras import regularizers
from tensorflow.python.util import nest
from tensorflow.python.ops import array_ops

# Import variables and functions from my own scripts
from functions import f1, plot_history, arr_replacevalue
from load_features import (
    train_features_AW2,
    val_features_AW2,
    train_labels_AW2,
    val_labels_AW2,
    labels_reshaper,
    features_reshaper,
)

%matplotlib inline

# Limit GPU memory usage
for device in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(device, True)

# Prepare data

In [ ]:
# Reshape data to specified sequence length
length = 60
seq_train_features = features_reshaper(train_features_AW2, length)
seq_val_features = features_reshaper(val_features_AW2, length)

seq_train_labels = labels_reshaper(train_labels_AW2, length)
seq_val_labels = labels_reshaper(val_labels_AW2, length)

In [ ]:
def comp_sampleweights(labels):
    # Convert one-hot encoded labels back to label integers
    train_label_ints = np.argmax(labels, axis=2)

    # Compute class weights with sklearn
    class_weights = class_weight.compute_class_weight(
        "balanced", np.unique(train_label_ints), train_label_ints.flatten()
    )
    d_class_weights = dict(enumerate(class_weights))

    # Copy label integer array
    arr = train_label_ints.copy()

    # Pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample
    return arr_replacevalue(arr, d_class_weights)
    
train_samples_weights = comp_sampleweights(seq_train_labels) 

## Possibilities for custom FW-RNN building
-  Build a class fw_layer first to define the inner computation block
-  Build the FW-RNN model class to define the outer model (which will be trained)

# Best option:
-  Build custom FW_RNN cell and wrap it in RNN(FW_RNN)
    -  The cell abstraction, together with the generic keras.layers.RNN class, make it very easy to implement custom RNN architectures for your research.


# Create FW-RNN Cell
Created by using this guide: https://www.tensorflow.org/guide/keras/custom_layers_and_models

In [ ]:
# Source: https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/layers/recurrent.py#L1218-L1416
        
def _generate_zero_filled_state_for_cell(cell, inputs, batch_size, dtype):
    if inputs is not None:
        batch_size = array_ops.shape(inputs)[0]
        dtype = inputs.dtype
    return _generate_zero_filled_state(batch_size, cell.state_size, dtype)

def _generate_zero_filled_state(batch_size_tensor, state_size, dtype):
    """Generate a zero filled tensor with shape [batch_size, state_size]."""
    if batch_size_tensor is None or dtype is None:
        raise ValueError(
            'batch_size and dtype cannot be None while constructing initial state: '
            'batch_size={}, dtype={}'.format(batch_size_tensor, dtype))

    def create_zeros(unnested_state_size):
        flat_dims = tensor_shape.TensorShape(unnested_state_size).as_list()
        init_state_size = [batch_size_tensor] + flat_dims
        return array_ops.zeros(init_state_size, dtype=dtype)

    if nest.is_nested(state_size):
        return nest.map_structure(create_zeros, state_size)
    else:
        return create_zeros(state_size)


In [ ]:
class FW_RNNCell(layers.Layer):
    def __init__(self, units, use_bias, activation, step, **kwargs):
        super(FW_RNNCell, self).__init__(**kwargs)
        self.units = units
        self.step = step
        self.use_bias = use_bias
        self.activation = activations.get(activation)

        self.state_size = self.units
        self.output_size = self.units

        # Initializer for the kernel weights matrix, used for the linear transformation of the inputs
        self.kernel_initializer = initializers.get("glorot_uniform")

        # Initializer for the bias vector.
        self.bias_initializer = initializers.get("zeros")

        # Initializer for the recurrent_kernel (hidden) weights matrix, used for the linear
        # transformation of the recurrent state.
        self.recurrent_initializer = initializers.get("identity")

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            name="kernel",
            initializer=self.kernel_initializer,
        )
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            name="recurrent_kernel",
            initializer=self.recurrent_initializer,
        )
        if self.use_bias:
            self.bias = self.add_weight(
                shape=(self.units,), name="bias", initializer=self.bias_initializer,
            )
        else:
            self.bias = None
        self.built = True

    def call(self, inputs, states, training=None):
        prev_output = states[0] if nest.is_sequence(states) else states

        h = K.dot(inputs, self.kernel)
        if self.bias is not None:
            h = K.bias_add(h, self.bias)

        output = h + K.dot(prev_output, self.recurrent_kernel)
        if self.activation is not None:
            output = self.activation(output)

        new_state = [output] if nest.is_sequence(states) else output
        return output, new_state

    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        return _generate_zero_filled_state_for_cell(self, inputs, batch_size, dtype)

#     def get_config(self):
#         config = {
#             "units": self.units,
#             "activation": activations.serialize(self.activation),
#             "use_bias": self.use_bias,
#             "kernel_initializer": initializers.serialize(self.kernel_initializer),
#             "recurrent_initializer": initializers.serialize(self.recurrent_initializer),
#         }

# Build model

In [ ]:
def build_model():
    model = Sequential(name="FW-RNN")
    model.add(layers.InputLayer(input_shape= (seq_train_features.shape[1], seq_train_features.shape[2])))
    model.add(
        layers.RNN(
            FW_RNNCell(units=512, use_bias=True, activation="tanh", step=1),
            return_sequences=True,
            name = 'FW-RNN'
        )
    )
    model.add(layers.LayerNormalization())
    model.add(layers.Dense(7, activation="softmax", name="Dense_Output"))
    model.compile(
        optimizer="adagrad",
        loss=CategoricalCrossentropy(label_smoothing=0.1),
        metrics=["accuracy", f1, "AUC"],
    )
    return model

In [ ]:
fw_rnn = build_model()
fw_rnn.summary()

# Train + Evaluate model

In [ ]:
# AW2_norm_minitrain = AW2_norm_minitrain.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
history_best = fw_rnn.fit(
    seq_train_features,
    seq_train_labels,
    sample_weight=train_samples_weights,
    validation_data=(seq_val_features, seq_val_labels),
    epochs=50,
    verbose=2,
)

In [ ]:
# class fw_layer(layers.Layer):
#     def __init__(
#         self, units=32, input_shape=(params.sequence_dim, params.features_dim)
#     ):
#         # A layer encapsulates both a state (the layer's "weights")
#         super(fw_layers, self)
#         # Define weight initializers of input weights and input biases
#         W_init = tf.keras.initializers.GlorotUniform()
#         b_init = tf.keras.initializers.zeros()

#     def call(self, inputs):
#         #         a transformation from inputs to outputs (a "call", the layer's forward pass)
#         return

In [ ]:
# class fw_model(tf.keras.Model):
#     def __init__(self, **kwargs):
#         super(CustomModel, self).__init__(**kwargs)

        
#     def call(self, inputs):
#         x = self.

In [ ]:
# ops.executing_eagerly_outside_functions()
# from tensorflow.python.distribute import distribution_strategy_context as ds_context
# from tensorflow.python.eager import context
# from tensorflow.python.keras.engine.base_layer import Layer
# from tensorflow.python.keras.engine.input_spec import InputSpec
# from tensorflow.python.keras.saving.saved_model import layer_serialization
# from tensorflow.python.keras.utils import control_flow_util
# from tensorflow.python.keras.utils import generic_utils
# from tensorflow.python.keras.utils import tf_utils
# from tensorflow.python.ops import control_flow_ops
# from tensorflow.python.ops import math_ops
# from tensorflow.python.ops import state_ops
# from tensorflow.python.platform import tf_logging as logging
# from tensorflow.python.training.tracking import base as trackable
# from tensorflow.python.training.tracking import data_structures
# from tensorflow.python.util.tf_export import keras_export
# from tensorflow.tools.docs import doc_controls